In [9]:
# bob.refresh {"cron": "*/30 * * * *", "timeout": 40000}

In [10]:
import bob
hc = bob.HealthCheck()
hc_key = "608ba515-2285-45ad-92d4-47491493ac12"
start = hc.start(hc_key)
print(start)

HealthCheck init in prod mode
Start ==> send to https://health.cashstory.com/ping/608ba515-2285-45ad-92d4-47491493ac12, 2020-04-28


In [11]:
!pip install --upgrade google-api-python-client oauth2client

Requirement already up-to-date: google-api-python-client in /home/ftp/.local/lib/python3.7/site-packages (1.8.2)
Requirement already up-to-date: oauth2client in /home/ftp/.local/lib/python3.7/site-packages (4.1.3)


In [12]:
%run __init__.ipynb

Successfully connected to MongoDB


In [13]:
"""Hello Analytics Reporting API V4."""

from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials

SCOPES = ['https://www.googleapis.com/auth/analytics.readonly']
KEY_FILE_LOCATION = param_folder + 'analytics_id_read.json'
VIEW_ID = '214312751'
METRICS = [{'expression': 'ga:users'}]
# metrics = [{'expression': 'ga:sessions'}]
DIMENTIONS = [{'name': 'ga:country'}]
FIRSTDATA = 30

def initialize_analyticsreporting():
  """Initializes an Analytics Reporting API V4 service object.

  Returns:
    An authorized Analytics Reporting API V4 service object.
  """
  credentials = ServiceAccountCredentials.from_json_keyfile_name(
      KEY_FILE_LOCATION, SCOPES)

  # Build the service object.
  analytics = build('analyticsreporting', 'v4', credentials=credentials)

  return analytics

analytics = initialize_analyticsreporting()

In [14]:
import pandas as pd
from datetime import datetime, timedelta, date
from dateutil.rrule import rrule, DAILY
from time import sleep
from googleapiclient.discovery import build
from oauth2client.service_account import ServiceAccountCredentials
import time

now = datetime.now().strftime('%d/%m/%Y %H:%M:%S')

def convert_reponse_to_df(response, date=None):
      list = []
      # parse report data
      for report in response.get('reports', []):

        columnHeader = report.get('columnHeader', {})
        dimensionHeaders = columnHeader.get('dimensions', [])
        metricHeaders = columnHeader.get('metricHeader', {}).get('metricHeaderEntries', [])
        rows = report.get('data', {}).get('rows', [])

        for row in rows:
            dict = {}
            dimensions = row.get('dimensions', [])
            dateRangeValues = row.get('metrics', [])

            for header, dimension in zip(dimensionHeaders, dimensions):
                dict[header] = dimension

            for i, values in enumerate(dateRangeValues):
                for metric, value in zip(metricHeaders, values.get('values')):
                    if ',' in value or ',' in value:
                        dict[metric.get('name')] = float(value)
                    else:
                        dict[metric.get('name')] = int(value)
                    if date is not None:
                        dict['date'] = date
            list.append(dict)

        df = pd.DataFrame(list)
        return df


def get_report(analytics, start_date, end_date, view_id, metrics, dimensions, dimensionFilterClauses=[], segments=[]):
    body = {
        'reportRequests': [
        {
          'viewId': view_id,
          'dateRanges': [{'startDate':start_date, 'endDate': end_date}],
          'metrics': metrics,
          'dimensions': dimensions,
          'pageSize': 10000,
          'dimensionFilterClauses': dimensionFilterClauses,
          'segments': segments,
        }]
      }
#     print('body', body)
    return analytics.reports().batchGet(
      body=body
    ).execute()


def return_ga_data(start_date, end_date, view_id, metrics, dimensions, split_dates, group_by=[], dimensionFilterClauses=[], segments=[]):
    if split_dates == False:
        return convert_reponse_to_df(get_report(analytics, start_date, end_date, view_id, metrics, dimensions, dimensionFilterClauses, segments))
    else:
        start_date = datetime.strptime(start_date, '%Y-%m-%d').date()
        end_date = datetime.strptime(end_date, '%Y-%m-%d').date()

        df_total = pd.DataFrame()
        for date in rrule(freq=DAILY, dtstart=start_date, until=end_date):
            date = str(date.date())
            df_total = df_total.append(convert_reponse_to_df(get_report(analytics, date, date, view_id, metrics, dimensions, dimensionFilterClauses, segments), date))

        if len(group_by) != 0:
            df_total = df_total.groupby(group_by).sum()

        return df_total

thirtydaysAgo = (datetime.now() - timedelta(FIRSTDATA)).strftime("%Y-%m-%d")
today = datetime.now().strftime("%Y-%m-%d")

start_time = time.time()
df = return_ga_data(thirtydaysAgo, today, VIEW_ID, METRICS, DIMENTIONS, True)
print("Script execution completed at " + now + ". Time: --- %s secnds ---" % (time.time() - start_time))

Script execution completed at 28/04/2020 06:51:46. Time: --- 4.229133367538452 secnds ---


In [15]:
def domain_301(df):
    cols_to_rename = {"ga:country": "ENTITY", "date": "DATE", "ga:users": "VALUE_D"}
    domain = df.copy().rename(index=str, columns=cols_to_rename).fillna("Not defined")
    domain['DATE'] = pd.to_datetime(domain['DATE'], format='%Y-%m-%d').dt.strftime('%d/%m/%Y')
    
    #-- Consolidate data
    domain_ww = domain.copy()
    domain_ww['ENTITY'] = 'WORLDWIDE'
    
    # Concat & groupby
    domain_final = pd.concat([domain,domain_ww], axis=0)
    domain_final = domain_final.groupby(['ENTITY','DATE'], as_index=False).agg({'VALUE_D': 'sum'})
    domain_final = domain_final.sort_values(["DATE"], ascending=True).reset_index(drop=True)
    domain_final['VALUE'] = domain_final.groupby(['ENTITY'], as_index=True).agg({'VALUE_D': 'cumsum'})
    
    domain_final['DATE_ORDER'] = pd.to_datetime(domain_final['DATE'], format='%d/%m/%Y').dt.strftime('%Y%m%d')
    domain_final['LAST_UPDATE'] = datetime.now().strftime('%d/%m/%Y %H:%M:%S')
    return domain_final

domain301 = domain_301(df)
if use_mongo:
    bob.mongo.save_df(domain301,'301',db_app,True)
domain301

Dataframe 301 successfully save in database app-wsr-dev in MongoDB. Time: --- 0.04262685775756836 secnds ---


,ENTITY,DATE,VALUE_D,VALUE,DATE_ORDER,LAST_UPDATE
0,(not set),01/04/2020,2,2,20200401,28/04/2020 06:51:50
1,Ukraine,01/04/2020,1,1,20200401,28/04/2020 06:51:50
2,Italy,01/04/2020,27,27,20200401,28/04/2020 06:51:50
3,Argentina,01/04/2020,1,1,20200401,28/04/2020 06:51:50
4,Togo,01/04/2020,1,1,20200401,28/04/2020 06:51:50
...,...,...,...,...,...,...
858,Mayotte,31/03/2020,1,2,20200331,28/04/2020 06:51:50
859,Réunion,31/03/2020,2,37,20200331,28/04/2020 06:51:50
860,Madagascar,31/03/2020,1,5,20200331,28/04/2020 06:51:50
861,United Arab Emirates,31/03/2020,1,58,20200331,28/04/2020 06:51:50


In [16]:
done = hc.done(hc_key)
print(done)

End ==> send to https://health.cashstory.com/ping/608ba515-2285-45ad-92d4-47491493ac12, 2020-04-28
